In [1]:
#LangChain에서 지원하는 Vector Store
#=> https://python.langchain.com/v0.1/docs/integrations/vectorstores/

#Chroma
# 크로마는 오픈소스 벡터 데이터베이스입니다. Chroma는 Apache 2.0 라이선스가 부여됩니다.
# => 공식도큐먼트 : https://docs.trychroma.com/
# => 홈페이지 :  https://www.trychroma.com/

# 경고 메시지 무시
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[OPENAI_API_KEY]\n{os.environ['OPENAI_API_KEY']}\n")
print(f"[HUGGINGFACEHUB_API_TOKEN]\n{os.environ['HUGGINGFACEHUB_API_TOKEN']}")

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

with open("../data/04.급여규정_16.06.01.txt") as f:
    file = f.read()
print(f'*type:{type(file)}')


text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    # 청크 간의 중복되는 문자 수를 설정합니다.
    chunk_overlap=50,
    # 문자열 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자로 정규식을 사용할지 여부를 설정합니다.
    is_separator_regex=False,
)

# text_splitter 이용해서 file 텍스트를 문서단위로 출력
# -> separator를 지정하지 않았으므로, 문서에서 기본값 "\n\n" 로 구분해서 나눔.
split_docs = text_splitter.create_documents([file])
print(f'*문서 분할 수: {len(split_docs)}')


*type:<class 'str'>
*문서 분할 수: 17


In [4]:
# embedding 모델 생성 
# => 아래처럼 huggingface 모델과 chroma_db 연동할때 에러남. 
# => InvalidDimensionException: Embedding dimension 768 does not match collection dimensionality 1536
# => 확인결과 : text-embedding-ada-002 모델이 차원이 1536인데 이것만 지원되도록 된건지 확인이 필요함.

# ./cache/ 경로에 다운로드 받도록 설정
os.environ["HF_HOME"] = "./cache/"

from langchain_community.embeddings import (
    HuggingFaceEmbeddings,
    HuggingFaceBgeEmbeddings,
)

# model_name = 없으면: 기본 'sentence-transformers/all-mpnet-base-v2' 모델임.
# embeddings = HuggingFaceEmbeddings()  # HuggingFace 임베딩을 생성합니다.

# InvalidDimensionException: Embedding dimension 128 does not match collection dimensionality 1536
embeddings = HuggingFaceEmbeddings(
    model_name="bongsoo/kpf-sbert-128d-v1", 
    show_progress=True
)

print(f'*임베딩 모델: {embeddings.model_name}')

#embeddings=OpenAIEmbeddings()
#print(f'*임베딩 모델: {embeddings.model}')

text = "임베딩 테스트를 하기 위한 샘플 문장입니다."  # 테스트용 문서 텍스트를 정의합니다.
query_result = embeddings.embed_query(text)
print(f'*벡터길이:{len(query_result)}')

*임베딩 모델: bongsoo/kpf-sbert-128d-v1


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.38it/s]

*벡터길이:128


In [5]:
# 벡터 저장소에 임베딩 하여 저장
%time chroma_db = Chroma.from_documents(documents=split_docs, embedding=embeddings)

Batches: 100%|██████████| 1/1 [00:01<00:00,  1.92s/it]

CPU times: user 15.6 s, sys: 462 ms, total: 16.1 s
Wall time: 2.69 s


In [6]:
# query 에 질문을 담아서 db 에서 유사한 문장을 찾아냅니다.

# 유사도 검색(쿼리)
similar_docs = chroma_db.similarity_search("신규 입사자 급여는 얼마?")
print(f"*검색된 docs 계수: {len(similar_docs)}")

for i, similar_doc in enumerate(similar_docs):
    print(f"Doc {i} : {similar_doc}\n")


Batches: 100%|██████████| 1/1 [00:00<00:00, 16.98it/s]

*검색된 docs 계수: 4
Doc 0 : page_content='7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금\n8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. \n8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.\n8.3 상여급 지급기준(액)은 별도로 정한다.\n8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.\n9. 퇴직금\n9.1 지급대상'

Doc 1 : page_content='10. 부록\n10.1시행일\n- 이 규정은 2007년 9월 14일부터 제정·시행한다.\n- 이 규정은 2016년 6월 1일부터 개정, 시행한다.\n급여규정 1/3'

Doc 2 : page_content='을 공제한 금액으로 한다.\n5.8.3 징집명령에 의한 휴직자에 대하여는 휴직발령일로부터 복직발령일 전일까지 급여를 지급하지 \n아니한다.\n5.8.4 기타 개인적인 사정으로 휴직하였을 경우 그 기간동안의 급여는 지급하지 아니한다.\n5.9 퇴직자 급여\n5.9.1 퇴직자는 퇴직명령전일까지의 급여를 일할계산한다. .\n5.9.2 급여 지급일 전 최소 1개월 이전에 퇴직의사를 밝히지 않아 발생한 퇴직 당월의 과지급'

Doc 3 : page_content='6.6.1 직원이 상위직급으로 승진한 경우 승진된 직급의 초임을 적용함을 원칙으로 한다.\n6.6.2 승진후 월급여가 승진전 월급여보다 적은 경우에는 월급여는 승진된 직급의 초임을 적용하되 \n그 차액은 조정수당으로 지급한다. \n단, 임금인상, 승급 등에 의한 그 차액의 감소분만큼은 조정수당을 차감, 조정한다.\n7. 제수당\n필요시 수당을 별도로 정하여 지급할 수 있다.\n8. 상여금'



In [8]:
# embedding_Vector 를 통해 유사도 검색
#embedding_vector = OpenAIEmbeddings().embed_query("신규 입사자 급여는 얼마?")
text = "임베딩 테스트를 하기 위한 샘플 문장입니다."  # 테스트용 문서 텍스트를 정의합니다.
embedding_vector = embeddings.embed_query(text)
print(f"*백터: {embedding_vector[:5]}")

similar_docs = chroma_db.similarity_search_by_vector(embedding_vector)
print(f"*검색된 docs 계수: {len(similar_docs)}")

for i, similar_doc in enumerate(similar_docs):
    print(f"Doc {i} : {similar_doc}\n")

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.17it/s]

*백터: [-0.14662310481071472, 0.1815720945596695, -0.11101395636796951, 0.06866949051618576, -0.20160073041915894]
*검색된 docs 계수: 4
Doc 0 : page_content='3.2 일반적으로 월급여라 함은 기본급과 제수당을 합한 금액을 말한다.\n4. 급여조정\n급여의 인상은 년 1회 1월 1일부로 실시함을 원칙으로 한다.\n5. 계산과 지급\n5.1 계산기간\n급여계산기간은 매월 1일부터 당월 말일까지로 한다.\n5.2 일할계산\n5.2.1 급여의 변경이 있는 월의 급여는 일할계산한다.\n5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수'

Doc 1 : page_content='5.2.2 일할계산은 월의 대소에 관계없이 월급여의 1/30로 한다.\n5.3 일할일수\n일할계산에 있어 그 일할일수는 실근로일수를 기준으로 한다.\n5.4 계산단위\n급여계산은 원단위에서 이를 절사한다.\n5.5 급여지급일\n5.5.1 급여지급일은 매월 20일로 한다.\n단, 지급일이 휴일인 경우에는 그 전일로 한다.\n5.5.2 비상재해, 기타 부득이한 사유가 발생시에는 지급일을 변경할 수 있다\n5.6 지급방법'

Doc 2 : page_content='10. 부록\n10.1시행일\n- 이 규정은 2007년 9월 14일부터 제정·시행한다.\n- 이 규정은 2016년 6월 1일부터 개정, 시행한다.\n급여규정 1/3'

Doc 3 : page_content='6.4.2 당해 승급기준기간 중 인사고과성적이 각 직급별 하위10%이내인 자\n6.4.3 당해 승급기준기간 중 휴직기간이 3개월 초과자. 단, 공상자는 제외\n6.5 승급보류\n정기 승급시 휴직인 자는 휴직기간 동안 승급을 보류하고 복직 후 6.2 내지 6.4의 규정에 의거 \n승급을 실시한다. 단, 공상자는 예외로 한다.\n6.6 승진자의 월급여\n6.6.1 직원이 상위직급으로 승진한 경우 

In [9]:
# VectorStoreRetreiver 는 벡터 저장소를 사용하여 문서를 검색하는 Retriever 입니다.
# 이는 벡터 저장소 클래스를 경량 래퍼로 감싸서 리트리버 인터페이스를 따르도록 하는 것입니다.
# 유사성 검색 및 MMR과 같이 벡터 저장소가 구현한 검색 방법을 사용하여 벡터 저장소의 텍스트를 쿼리합니다.
# 생성된 VectorStore 에 as_retriever() 함수로 반환받습니다.
    
# retriever 생성
retriever = chroma_db.as_retriever(search_kwargs={"k": 1})

# similarity_search 를 통해 유사도 높은 1개 문서를 검색
relevant_docs = retriever.get_relevant_documents("신규 입사자 급여는 얼마?")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
print(relevant_docs[0].page_content)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Batches: 100%|██████████| 1/1 [00:00<00:00, 15.18it/s]

문서의 개수: 1
[검색 결과]

7. 제수당
필요시 수당을 별도로 정하여 지급할 수 있다.
8. 상여금
8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. 
8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.
8.3 상여급 지급기준(액)은 별도로 정한다.
8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.
9. 퇴직금
9.1 지급대상


In [10]:
# search_type
# - search_type 매개변수에 검색 알고리즘을 지정할 수 있습니다.
# - similarity(기본값), mmr, similarity_score_threshold 등의 옵션을 지정할 수 있습니다.

#아래에서 지정 옵션 별 차이에 대해 다룹니다.
# - search_type='similarity'
# 기본 값으로 설정되어 있습니다. vector store 의 유사도 알고리즘 기반으로 상위 K개의 문서를 검색합니다.

retriever = chroma_db.as_retriever(search_type="similarity", search_kwargs={"k": 2})

relevant_docs = retriever.get_relevant_documents("신규 입사자 급여는 얼마?")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

Batches: 100%|██████████| 1/1 [00:00<00:00, 20.15it/s]

문서의 개수: 2
[검색 결과]

7. 제수당
필요시 수당을 별도로 정하여 지급할 수 있다.
8. 상여금
8.1 매년 경영성과등을 고려하여 특별 상여금을 지급할수 있다.. 
8.2 상여금은 지급일 현재 휴직자를 제외한 재직중인 자에 한하여 지급한다.
8.3 상여급 지급기준(액)은 별도로 정한다.
8.4 상여금은 발령일을 기준으로 금액을 일할 계산하여 지급한다. 이 때, 휴직기간은 일할 계산에서 제외한다.
9. 퇴직금
9.1 지급대상
10. 부록
10.1시행일
- 이 규정은 2007년 9월 14일부터 제정·시행한다.
- 이 규정은 2016년 6월 1일부터 개정, 시행한다.
급여규정 1/3


In [11]:
# Maximal Marginal Relevance(MMR) 검색
# MMR은 쿼리와 관련된 항목을 검색하면서 동시에 내용의 중복을 최소화하는 기법입니다.
# 이 방법은 단순히 관련성이 높은 항목들을 선택하는 대신, 관련성과 다양성 사이의 균형을 찾는 데 중점을 둡니다.
#
# MMR 에 대해 이해해보기
# 모임에 참석해 친구를 위해 새로운 사람들을 소개하는 상황을 상상해 보세요. 친구가 만나고 싶어하는 사람들의 특징(embedding vector)을 알고 있습니다.
# MMR 방식을 적용하면 다음과 같은 접근법을 사용합니다:
#
# 예) 모임에 있는 모든 사람들의 프로필(list of embedding vector)을 살펴봅니다.
# 1. 친구의 관심사와 특성에 부합하는 사람을 찾아 소개합니다.
# 2. 그 후, 다시 참여한 사람들을 살펴보되, 이번에는 이미 소개한 사람과는 다른 특성을 가진 사람을 찾습니다.
# 3. 여기서 lambda mult 매개변수는 친구의 취향과 새로운 특성 사이의 균형을 조정하는 역할을 합니다.
# 4. 이 과정을 친구가 만나길 원하는 사람들의 수(k 매개변수)에 도달할 때까지 반복합니다.
# 5. 마지막으로, 친구가 만난 사람들의 목록을 제공합니다.
# 6. 이 예시를 통해 MMR이 관련성 높은 항목을 선택 하는 동시에 내용의 다양성을 유지 하려는 방식을 이해할 수 있습니다.
#
# search_type='mmr' 을 통해 MMR 을 사용한 검색을 수행할 수 있습니다.

retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2})

relevant_docs = retriever.get_relevant_documents("신규 입사자 급여는 얼마?")
print(f"문서의 개수: {len(relevant_docs)}")

print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)
    

Batches: 100%|██████████| 1/1 [00:00<00:00, 19.01it/s]
Number of requested results 20 is greater than number of elements in index 17, updating n_results = 17


문서의 개수: 2
[검색 결과]

9. 퇴직금
9.1 지급대상
퇴직금은 만 1년이상 근속한 직원이 퇴직하였을 경우에 지급한다. 
9.2 근속기간
9.2.1 근속기간은 입사일로부터 퇴직전일까지로 한다
9.2.2 근속기간은 년, 월, 일 단위로 계산, (근속연수 × 365) + (근속월수 × 30) + 근속일수에 의
해 일수로 계산한다.
9.3 퇴직금의 산정방법
9.3.1 퇴직금은 근속년수 1년에 대하여 평균임금의 30일분을 지급한다.
5.13 전임·파견, 복귀자 급여
전임·파견, 복귀자 급여는 발령일을 기준으로 매월 20일까지는 신근무지에서, 21일 이후에는 전근
무지에서 지급한다.
5.14 휴업지불
회사의 귀책사유로 인하여 휴업하는 경우에는 평균임금의 70%이상을 휴업수당으로 지급한다.
단, 노동위원회의 승인을 얻은 경우에는 그 범위 이하의 휴업지불을 할 수 있다.
5.15 공제
급여의 지급에 있어 다음 사항에 해당하는 것은 이를 공제할 수 있다.


In [16]:
# search_type='similarity_score_threshold' 
# - score_threshold 기준을 충족하는 유사도 문서가 반환됩니다.
# 만약 {"k": 3} 이지만, {"score_threshold": 0.5} 로 설정되었는데 score 가 0.5를 넘는 문서가 2개 밖에 없다면, 
# 결과는 3개 문서가 아닌 2개 문서가 반환됩니다.

retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 2, "score_threshold": 0.5},
)

relevant_docs = retriever.get_relevant_documents("신규 입사자 급여는 얼마?")
print(f"문서의 개수: {len(relevant_docs)}")

print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

Batches: 100%|██████████| 1/1 [00:00<00:00, 26.97it/s]

문서의 개수: 0
[검색 결과]

